In [1]:
import shap

In [1]:
import io
import sys
from autogluon.tabular import TabularDataset, TabularPredictor
import json
import pandas as pd
def main():
    # 표준 입력으로부터 JSON 데이터 읽기
    input_data = '''
    {
        "admi_cty_no": [41590590],
        "card_tpbuz_cd": ["D01"],
        "amt": [5],
        "cnt": [906],
        "TOTAL_POPULATION": [7667999.44],
        "운영점포평균영업기간": [37.7],
        "폐업점포평균영업기간": [33.8],
        "상권변동지표구분": ["LL"]
    }'''

    try:
        data = json.loads(input_data)

        # JSON 데이터를 DataFrame으로 변환
        df_predict = pd.DataFrame(data)
        df_predict = TabularDataset(df_predict)
        # 운영점포평균영업기간의 값만 조회
        # operating_avg_period = df_predict['store_avg_period'].values[0]
        predictor2 = TabularPredictor.load(r"AutogluonModels\ag-20240715_073451")
        y_pred = predictor2.predict(df_predict.drop(columns=['amt']))
        
        # 결과를 JSON 형태로 반환
        result = {
            "status": "success",
            "actual_value": 365433405,  # 실제값 (예시로 입력)
            "predicted_value": y_pred.iloc[0]  # numpy.int64 -> 기본 데이터 타입으로 변환
        }
    except Exception as e:
        result = {
            "status": "error",
            "message": str(e)
        }

    # 결과를 JSON 형태로 표준 출력
    # sys.stdout = io.TextIOWrapper(sys.stdout.buffer, encoding='utf-8')
    print(json.dumps(result, ensure_ascii=False))

if __name__ == "__main__":
    main()

TypeError: Object of type float32 is not JSON serializable

In [7]:
import json
import pandas as pd
from autogluon.tabular import TabularDataset, TabularPredictor
import os

def main():
    # 표준 입력으로부터 JSON 데이터 읽기
    input_data = '''
    {
        "admi_cty_no": [41590590],
        "card_tpbuz_cd": ["D01"],
        "amt": [5],
        "cnt": [906],
        "TOTAL_POPULATION": [7667999.44],
        "store_avg_period": [37.7],
        "closure_avg_period": [33.8],
        "business_change_indicator": ["LL"]
    }'''

    try:
        data = json.loads(input_data)

        # JSON 데이터를 DataFrame으로 변환
        df_predict = pd.DataFrame(data)
        df_predict = df_predict.rename(columns={
            'store_avg_period': '운영점포평균영업기간',
            'closure_avg_period': '폐업점포평균영업기간',
            'business_change_indicator': '상권변동지표구분'
        })
        df_predict = TabularDataset(df_predict)
        
        # 운영점포평균영업기간의 값만 조회
        # operating_avg_period = df_predict['store_avg_period'].values[0]

        # 모델 로드
        model_path = r"AutogluonModels\ag-20240715_073451"
        if not os.path.exists(model_path):
            raise FileNotFoundError(f"모델 경로를 찾을 수 없습니다: {model_path}")
        
        predictor2 = TabularPredictor.load(model_path)
        
        # 'amt' 열을 제외하고 예측
        y_pred = predictor2.predict(df_predict.drop(columns=['amt']))
        
        # 예측 값을 float으로 변환
        predicted_value = float(y_pred.iloc[0])
        
        # 결과를 JSON 형태로 반환
        result = {
            "status": "success",
            "actual_value": 365433405,  # 실제값 (예시로 입력)
            "predicted_value": predicted_value  # numpy.float32 -> float 변환
        }
    except Exception as e:
        result = {
            "status": "error",
            "message": str(e)
        }

    # 결과를 JSON 형태로 표준 출력
    print(json.dumps(result, ensure_ascii=False))

if __name__ == "__main__":
    main()

{"status": "success", "actual_value": 365433405, "predicted_value": 339639264.0}
